In [2]:
from pyspark import SparkConf, SparkContext
import pandas as pd

In [29]:
df = pd.read_csv('1800.csv', header=None)

In [31]:
df.head()

,0,1,2,3,4,5,6,7
0,ITE00100554,18000101,TMAX,-75,NaN,NaN,E,NaN
1,ITE00100554,18000101,TMIN,-148,NaN,NaN,E,NaN
2,GM000010962,18000101,PRCP,0,NaN,NaN,E,NaN
3,EZE00100082,18000101,TMAX,-86,NaN,NaN,E,NaN
4,EZE00100082,18000101,TMIN,-135,NaN,NaN,E,NaN


In [34]:
df.drop(columns=[4,5,6,7], axis=1)

,0,1,2,3
0,ITE00100554,18000101,TMAX,-75
1,ITE00100554,18000101,TMIN,-148
2,GM000010962,18000101,PRCP,0
3,EZE00100082,18000101,TMAX,-86
4,EZE00100082,18000101,TMIN,-135
5,ITE00100554,18000102,TMAX,-60
6,ITE00100554,18000102,TMIN,-125
7,GM000010962,18000102,PRCP,0
8,EZE00100082,18000102,TMAX,-44
9,EZE00100082,18000102,TMIN,-130


In [35]:
conf = SparkConf().setMaster("local").setAppName("min-temprature")

In [6]:
sc = SparkContext(conf=conf)

In [58]:
def parseLine(line):
    fields = line.split(',')
    stationID = fields[0]
    entryType= fields[2]
    temperature = float(fields[3]*0.1*(9.0/5.0) + 32)  # converting celcuis to forenheit
    return (stationID, entryType, temperature)

In [59]:
lines = sc.textFile("1800.csv")

parsedLines = lines.map(parseLine)

#only keep rows with TMIN
minTemps = parsedLines.filter(lambda x: "TMIN" in x[1])

# keep only statinoID and temperature for each station
stationsTemps = minTemps.map(lambda x: (x[0], x[2]))

# find the minimum temperature for the entire year for each station:
minTemps = stationsTemps.reduceByKey(lambda x,y: min(x,y))

minTemps

In [71]:
results = minTemps.collect();
for result in results:
    print(result[0] + "\t{:.2f}F".format(result[1]))

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 14.0 failed 1 times, most recent failure: Lost task 0.0 in stage 14.0 (TID 7, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "c:\spark\python\lib\pyspark.zip\pyspark\worker.py", line 229, in main
  File "c:\spark\python\lib\pyspark.zip\pyspark\worker.py", line 224, in process
  File "c:\users\mojiway\appdata\local\programs\python\python36-32\lib\site-packages\pyspark\rdd.py", line 2438, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "c:\users\mojiway\appdata\local\programs\python\python36-32\lib\site-packages\pyspark\rdd.py", line 2438, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "c:\users\mojiway\appdata\local\programs\python\python36-32\lib\site-packages\pyspark\rdd.py", l

In [ ]:
#the code is correct but I dont know why it doesnt work in jupyter notebook. I read somewhere that spark 2.* doesnt work with python 3.6